In [1]:
from graphing_libraries import ThreeDPlotter, TimeSeriesPlotter, TimeSeriesVisualizer
from data_generators import data_for_3d
import ipywidgets as widgets

from functools import partial
from IPython.display import display, clear_output

import pandas as pd

In [2]:
df = data_for_3d()
scale_factor_df = pd.DataFrame({
    'scale': ["scale1", "scale2", "scale3", "scale4",""],
    'factor': [0.5, 1.0, 1.5, 2.0,0.0]
})
scale_factor_df = scale_factor_df.set_index("scale")

In [ ]:

class InteractiveTimeSeriesPlot:
    def __init__(self, df, scale_factor_df):
        self.df = df
        self.scale_factor_df = scale_factor_df
        self.output = widgets.Output()

        self.columns_subset_widget = widgets.SelectMultiple(
            options=df.columns.tolist(),
            value=['Column1', 'Column2'],
            description='Columns'
        )

        default_years_subset = df.index.unique()[:4].tolist()
        self.years_subset_widget = widgets.SelectMultiple(
            options=df.index.unique().tolist(),
            value=default_years_subset,
            description='Years'
        )

        self.scale_factor_widget = widgets.FloatSlider(
            value=0.02,
            min=0.0,
            max=1.0,
            step=0.01,
            description='Scale Factor'
        )

        self.show_original_widget = widgets.Checkbox(
            value=True,
            description='Show Original'
        )

        self.growth_rate_widget = widgets.FloatSlider(
            value=0.02,
            min=0.0,
            max=1.0,
            step=0.01,
            description='Growth Rate'
        )

        self.moving_average_widget = widgets.Checkbox(
            value=False,
            description='Show Moving Average'
        )

        self.moving_average_window_widget = widgets.IntSlider(
            value=3,
            min=1,
            max=3,
            description='Moving Average Window'
        )

        self.plot_options_widgets = [
            widgets.Checkbox(value=False, description='Original + Moving Average'),
            widgets.Checkbox(value=False, description='Scaled on Original'),
            widgets.Checkbox(value=True, description='Growth Rate Moving Average'),
            widgets.Checkbox(value=True, description='Growth Rate on Original')
        ]

        self.apply_conversion_widget = widgets.Checkbox(
            value=False,
            description='Apply Conversion'
        )

        self.convert_series_widget = widgets.Dropdown(
            options=scale_factor_df.index.tolist(),
            value="",  # Default value is an empty string
            description='Convert Series',
            style={'description_width': 'initial'}
        )

        self.update_button = widgets.Button(description='Update Plot')
        self.update_button.on_click(self.update_plot)

        # Set up the event handler for the widget changes
        self.columns_subset_widget.observe(self.update_plot, names='value')
        self.years_subset_widget.observe(self.update_plot, names='value')
        self.scale_factor_widget.observe(self.update_plot, names='value')
        self.show_original_widget.observe(self.update_plot, names='value')
        self.growth_rate_widget.observe(self.update_plot, names='value')
        self.moving_average_widget.observe(self.update_plot, names='value')
        self.moving_average_window_widget.observe(self.update_plot, names='value')
        self.apply_conversion_widget.observe(self.update_plot, names='value')
        self.convert_series_widget.observe(self.update_plot, names='value')

        for option_widget in self.plot_options_widgets:
            option_widget.observe(self.update_plot, names='value')

        # Display the widgets and button
        display(self.columns_subset_widget, self.years_subset_widget, self.scale_factor_widget,
                self.show_original_widget, self.growth_rate_widget, self.moving_average_widget,
                self.moving_average_window_widget, *self.plot_options_widgets,
                self.apply_conversion_widget, self.convert_series_widget, self.update_button, self.output)

    def update_plot(self, change=None):
        with self.output:
            clear_output(wait=True)  # Clear previous output
            columns_subset = self.columns_subset_widget.value
            years_subset = self.years_subset_widget.value
            scale_factor = self.scale_factor_widget.value
            show_original = self.show_original_widget.value
            growth_rate = self.growth_rate_widget.value
            moving_average = self.moving_average_widget.value
            moving_average_window = self.moving_average_window_widget.value if moving_average else None
            plot_original_moving_average = self.plot_options_widgets[0].value
            plot_scaled_on_original = self.plot_options_widgets[1].value
            plot_growth_rate_moving_average = self.plot_options_widgets[2].value
            plot_growth_rate_on_original = self.plot_options_widgets[3].value
            apply_conversion = self.apply_conversion_widget.value
            convert_series = self.convert_series_widget.value

            # Update the DataFrame based on the selected scale factor
            updated_df = self.df.copy()
            factor = self.scale_factor_df.loc[convert_series]['factor'] if apply_conversion else 0.0
            if factor and (factor != 0.0):
                updated_df =  (updated_df * 1000) * (1 / factor)

            # Replace TimeSeriesPlotter with your actual class/method for plotting
            ts_plotter = TimeSeriesPlotter(updated_df)
            ts_plotter.plot_interactive_time_series(
                columns_subset=list(columns_subset),
                years_subset=list(years_subset),
                scale_factor=scale_factor,
                show_original=show_original,
                growth_rate=growth_rate,
                moving_average_window=moving_average_window,
                plot_original_moving_average=plot_original_moving_average,
                plot_scaled_on_original=plot_scaled_on_original,
                plot_growth_rate_moving_average=plot_growth_rate_moving_average,
                plot_growth_rate_on_original=plot_growth_rate_on_original
            )

# Example usage:
interactive_plot = InteractiveTimeSeriesPlot(df, scale_factor_df)


In [ ]:
###Overlayed Timeseries

In [ ]:
%matplotlib widget
ts_plotter = TimeSeriesPlotter(df)

class InteractiveTimeSeriesPlotter:
    def __init__(self, ts_plotter, df):
        self.ts_plotter = ts_plotter
        self.df = df

        # Get available columns and years from the DataFrame
        self.available_columns = df.columns.tolist()
        self.available_years = sorted(df.index.unique().tolist())

        # Initialize widgets
        self.columns_selector = widgets.SelectMultiple(
            options=self.available_columns,
            value=[self.available_columns[0]],
            description='Select Columns'
        )
        self.years_selector = widgets.SelectMultiple(
            options=self.available_years,
            value=[self.available_years[0]],
            description='Select Years'
        )
        self.growth_rates_sliders = {col: widgets.FloatSlider(value=0.0, min=0.0, max=5.0, step=0.01, description=f'Growth Rate ({col})')
                                     for col in self.available_columns}
        self.show_original_checkbox = widgets.Checkbox(value=True, description='Show Original')

        # Create button and output widgets
        self.plot_button = widgets.Button(description='Plot Time Series')
        self.output_plot = widgets.Output()

        # Set up event handlers
        self.plot_button.on_click(self.plot_time_series)

        # Display widgets
        display(self.columns_selector, self.years_selector)
        display(widgets.HBox(list(self.growth_rates_sliders.values())))
        display(self.show_original_checkbox)
        display(self.plot_button)
        display(self.output_plot)

    def plot_time_series(self, _):
        # Get selected values from widgets
        with self.output_plot:
            clear_output(wait=True)
        selected_columns = self.columns_selector.value
        selected_years = self.years_selector.value
        growth_rates_dict = {col: self.growth_rates_sliders[col].value for col in selected_columns}
        show_original = self.show_original_checkbox.value


        # Plot the time series
        with self.output_plot:
            self.ts_plotter.plot_multiple_time_series(
                columns_subset=selected_columns,
                years_subset=selected_years,
                growth_rates=growth_rates_dict,
                show_original=show_original
            )

# Assuming df and ts_plotter are already defined
interactive_plotter = InteractiveTimeSeriesPlotter(ts_plotter, df)


In [ ]:
# Assuming ts_plotter is an instance of TimeSeriesPlotter
%matplotlib widget
ts_plotter = TimeSeriesPlotter(df)

class TimeSeriesPlotterWithDistribution:
    def __init__(self, ts_plotter, df):
        self.ts_plotter = ts_plotter

        # Get available columns and years from the DataFrame (replace df with your actual DataFrame)
        self.available_columns = df.columns.tolist()
        self.available_years = sorted(df.index.unique().tolist())

        # Initialize widgets
        self.columns_selector = widgets.SelectMultiple(
            options=self.available_columns,
            value=[self.available_columns[0]],
            description='Select Columns'
        )
        self.years_selector = widgets.SelectMultiple(
            options=self.available_years,
            value=[self.available_years[0]],
            description='Select Years'
        )
        self.plot_selector = widgets.Select(
            options=["plot_time_series_with_distribution", "plot_area_under_curve","plot_heatmap"],
            value="plot_time_series_with_distribution",
            description='Select Plot Type'
        )
        self.growth_rate_slider = widgets.FloatSlider(
            value=0.05, min=0.0, max=1.0, step=0.01, description='Growth Rate'
        )
        self.plot_button = widgets.Button(description='Plot Time Series')
        self.output_plot = widgets.Output()

        # Set up event handlers
        self.plot_button.on_click(self.plot_time_series)

        # Display widgets
        display(self.columns_selector, self.years_selector, self.growth_rate_slider,self.plot_selector)
        display(self.plot_button)
        display(self.output_plot)

    def plot_time_series(self, _):
        # Get selected values from widgets
        with self.output_plot:
            clear_output(wait=True)
        selected_columns = self.columns_selector.value
        selected_years = self.years_selector.value
        growth_rate_to_apply = self.growth_rate_slider.value
        plot_type = self.plot_selector.value

        
        if plot_type == "plot_time_series_with_distribution":
            with self.output_plot:
                self.ts_plotter.plot_time_series_with_distribution(
                    column_subset=list(selected_columns),
                    year_subset=list(selected_years),
                    growth_rate=growth_rate_to_apply
                )
        elif plot_type == "plot_area_under_curve":
            with self.output_plot:
                self.ts_plotter.plot_area_under_curve(
                    column_subset=list(selected_columns),
                    year_subset=list(selected_years),
                    growth_rate=growth_rate_to_apply
                )
        elif plot_type == "plot_heatmap":
            with self.output_plot:
                self.ts_plotter.plot_heatmap(
                    column_subset=list(selected_columns),
                    year_subset=list(selected_years),
                    growth_rate=growth_rate_to_apply
                )

# Assuming df is your DataFrame and ts_plotter is already defined
interactive_plotter = TimeSeriesPlotterWithDistribution(ts_plotter, df)


In [3]:


class InteractivePlotter:
    def __init__(self, df, conversion_factor = ""):
        if conversion_factor:
            self.df = (df*1000)*(1/conversion_factor)
        else:
            self.df = df
        self.plotter = TimeSeriesVisualizer(df)

        # Get available columns and years from the dataframe
        self.available_columns = list(self.df.columns)
        self.available_years = list(self.df.index)

        # Create widgets
        self.column_selector = widgets.SelectMultiple(
            options=self.available_columns,
            value=[self.available_columns[0]],
            description='Columns'
        )

        self.year_selector = widgets.SelectMultiple(
            options=self.available_years,
            value=[self.available_years[0]],
            description='Years'
        )

        self.method_selector = widgets.Dropdown(
            options=self.get_plotter_methods(),
            value=self.get_plotter_methods()[0],
            description='Method'
        )

        self.growth_value_input = widgets.FloatText(
            value=0.05,
            description='Growth Value'
        )

        self.plot_button = widgets.Button(description='Plot')
        self.plot_button.on_click(self.plot)

        self.output_plot = widgets.Output()

        # Display widgets
        display(
            self.column_selector,
            self.year_selector,
            self.method_selector,
            self.growth_value_input,
            self.plot_button,
            self.output_plot
        )

    def get_plotter_methods(self):
        # Get the names of the methods in the plotter class
        return [method for method in dir(self.plotter) if callable(getattr(self.plotter, method)) and (not method.startswith("__")) and (not method.startswith("calculate"))]

    def plot(self, _):
        with self.output_plot:
            # Clear previous output
            clear_output(wait=True)

            # Get selected values from widgets
            selected_columns = list(self.column_selector.value)
            selected_years = list(self.year_selector.value)
            selected_method = self.method_selector.value
            growth_value = self.growth_value_input.value

            # Call the selected method with the chosen parameters
            method_to_call = getattr(self.plotter, selected_method)

            if selected_method in ["plot_horizontal_bar_subplots"]:
                method_to_call(columns=selected_columns, years=selected_years,  growth_rate=growth_value)
            else:
                method_to_call(columns=selected_columns, years=selected_years)

# Example usage
# Assuming df is your DataFrame and plotter is your TimeSeriesVisualizer
# plotter = TimeSeriesVisualizer(df)
interactive_plotter = InteractivePlotter(df)


SelectMultiple(description='Columns', index=(0,), options=('Column0', 'Column1', 'Column2', 'Column3', 'Column…

SelectMultiple(description='Years', index=(0,), options=(2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,…

Dropdown(description='Method', options=('plot_horizontal_bar_chart', 'plot_horizontal_bar_subplots', 'plot_ove…

FloatText(value=0.05, description='Growth Value')

Button(description='Plot', style=ButtonStyle())

Output()